In [1]:
print("ok")

ok


In [2]:
a=5+2
print(a)

7


#we have to load the Model


In [104]:
from langchain_groq import ChatGroq

In [105]:
from dotenv import load_dotenv

In [106]:
load_dotenv()

True

In [107]:
model=ChatGroq(model="qwen/qwen3-32b")

In [108]:
model.invoke("what is the capital of spain?").content

"<think>\nOkay, the user is asking for the capital of Spain. Let me start by recalling what I know. Spain is a country in southwestern Europe. I remember that major cities in Spain include Madrid, Barcelona, Valencia, and Seville. Out of these, I think Madrid is the capital. Wait, but I should double-check to be sure. Sometimes people confuse capitals with other large cities. For example, Barcelona is a big city, but I don't think it's the capital. Let me confirm.\n\nI can think of some other European capitals: Paris is France's, Berlin is Germany's, Rome is Italy's. Madrid is known for being the capital of Spain. Also, the Spanish government is based there, and it's the political and administrative center. The country's legislature, the Cortes Generales, meets in Madrid. There's also the Royal Palace of Madrid, which is the official residence of the Spanish royal family. Plus, international organizations often list Madrid as the capital. I don't recall any recent changes where the cap

In [109]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [110]:
embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

E0000 00:00:1760034147.198643 2673335 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760034147.296496 2673335 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [111]:
embedding_model.embed_query("whats the capital of spain?")

[0.02133907563984394,
 -0.026425788179039955,
 -0.0401422381401062,
 -0.00952532421797514,
 0.05966673046350479,
 0.008517565205693245,
 0.018477143719792366,
 -0.04099823534488678,
 0.026000509038567543,
 0.05848196893930435,
 0.006556855048984289,
 -0.01674470491707325,
 0.014796070754528046,
 0.023556547239422798,
 0.0020990693010389805,
 -0.00980911124497652,
 0.0015533780679106712,
 0.03306173160672188,
 0.03349010646343231,
 0.005182374734431505,
 0.034320104867219925,
 -0.011211013421416283,
 -0.03820530325174332,
 0.02922276221215725,
 0.005124237854033709,
 -0.047349244356155396,
 -0.0025375219993293285,
 -0.03829481080174446,
 0.0023374345619231462,
 -0.004464482422918081,
 -0.05245160683989525,
 0.04689197614789009,
 -0.022259138524532318,
 -0.0247244443744421,
 0.012250592932105064,
 -0.05921975523233414,
 -0.012469622306525707,
 0.01026015356183052,
 0.0013414594577625394,
 0.042398788034915924,
 0.012401831336319447,
 -0.015053754672408104,
 -0.08522633463144302,
 0.00973

# DATA Ingestion


In [112]:
from langchain.document_loaders import PyPDFLoader

In [113]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [114]:
import os

In [115]:
os.getcwd()

'/Users/siddhartha/Desktop/DOCUMENT_ANALYSIS/notebook'

In [116]:
file_path=os.path.join(os.getcwd(), "data", "sample.pdf")

In [117]:
loader=PyPDFLoader(file_path)

In [118]:
documents=loader.load()

In [119]:
documents

[Document(metadata={'producer': 'Adobe Acrobat Pro (64-bit) 25.1.20435', 'creator': 'Adobe Acrobat Pro (64-bit) 25.1.20435', 'creationdate': '2025-04-23T15:58:56-04:00', 'author': 'James  Aris', 'moddate': '2025-04-23T15:58:56-04:00', 'source': '/Users/siddhartha/Desktop/DOCUMENT_ANALYSIS/notebook/data/sample.pdf', 'total_pages': 75, 'page': 0, 'page_label': '1'}, page_content='Working Paper 25-021 \nGenerative AI and the Nature of Work \nManuel Hoffmann \nSam Boysel \nFrank Nagle \nSida Peng \nKevin Xu'),
 Document(metadata={'producer': 'Adobe Acrobat Pro (64-bit) 25.1.20435', 'creator': 'Adobe Acrobat Pro (64-bit) 25.1.20435', 'creationdate': '2025-04-23T15:58:56-04:00', 'author': 'James  Aris', 'moddate': '2025-04-23T15:58:56-04:00', 'source': '/Users/siddhartha/Desktop/DOCUMENT_ANALYSIS/notebook/data/sample.pdf', 'total_pages': 75, 'page': 1, 'page_label': '2'}, page_content='Generative AI and the Nature of Work \nManuel Hoffmann \nHarvard Business School \nSam Boysel \nHarvard Bus

In [120]:
len(documents)

75

# Chunking  This is experimental value , no detementic way to split the text

In [121]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len,
)

In [122]:
docs=text_splitter.split_documents(documents)

In [123]:
len(docs)

363

In [124]:
docs[0].metadata

{'producer': 'Adobe Acrobat Pro (64-bit) 25.1.20435',
 'creator': 'Adobe Acrobat Pro (64-bit) 25.1.20435',
 'creationdate': '2025-04-23T15:58:56-04:00',
 'author': 'James  Aris',
 'moddate': '2025-04-23T15:58:56-04:00',
 'source': '/Users/siddhartha/Desktop/DOCUMENT_ANALYSIS/notebook/data/sample.pdf',
 'total_pages': 75,
 'page': 0,
 'page_label': '1'}

In [125]:
docs[0].page_content

'Working Paper 25-021 \nGenerative AI and the Nature of Work \nManuel Hoffmann \nSam Boysel \nFrank Nagle \nSida Peng \nKevin Xu'

# Now vector Database we will use Faiss db (in memory)

In [126]:
from langchain.vectorstores import FAISS

In [127]:
vectorstore=FAISS.from_documents(docs, embedding_model)

In [128]:
relevant_doc=vectorstore.similarity_search("waht is gen ai?")

In [129]:
relevant_doc[0].page_content

'Zhang, 2023; Peng et al., 2023), it is less clear what the mechanisms behind these improvements\nare. Does the use of generative AI shift users to focus on particular types of tasks that lead to\n1Formerly known as Bard.\n1'

# This is a retriver process
means from vector db we are going to rank the most appropiate result

## Genaration of Output

In [130]:
retriver=vectorstore.as_retriever()

In [131]:
retriver.invoke("what is gen ai?")

[Document(id='80f96210-86d6-4e72-9097-7ae6cf8110ad', metadata={'producer': 'Adobe Acrobat Pro (64-bit) 25.1.20435', 'creator': 'Adobe Acrobat Pro (64-bit) 25.1.20435', 'creationdate': '2025-04-23T15:58:56-04:00', 'author': 'James  Aris', 'moddate': '2025-04-23T15:58:56-04:00', 'source': '/Users/siddhartha/Desktop/DOCUMENT_ANALYSIS/notebook/data/sample.pdf', 'total_pages': 75, 'page': 3, 'page_label': '4'}, page_content='Zhang, 2023; Peng et al., 2023), it is less clear what the mechanisms behind these improvements\nare. Does the use of generative AI shift users to focus on particular types of tasks that lead to\n1Formerly known as Bard.\n1'),
 Document(id='299d7473-52fc-4f5a-8f13-8140a8d336f5', metadata={'producer': 'Adobe Acrobat Pro (64-bit) 25.1.20435', 'creator': 'Adobe Acrobat Pro (64-bit) 25.1.20435', 'creationdate': '2025-04-23T15:58:56-04:00', 'author': 'James  Aris', 'moddate': '2025-04-23T15:58:56-04:00', 'source': '/Users/siddhartha/Desktop/DOCUMENT_ANALYSIS/notebook/data/sa

In [132]:
prompt_template="""
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

context:{context}
Question: {question}
Answer:"""

In [133]:
from langchain.prompts import PromptTemplate

In [134]:


prompt_template = PromptTemplate(
    template="Use this context: {context}, answer: {question}",
    input_variables=["context", "question"]
)

prompt = prompt_template  # This is the PromptTemplate object


In [135]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use this context: {context}, answer: {question}')

In [136]:
from langchain_core.output_parsers import StrOutputParser

In [137]:
parser=StrOutputParser()

In [138]:
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

In [139]:
from langchain_core.runnables import RunnablePassthrough, RunnableSequence, RunnableLambda

In [140]:

rag_chain=(
    {
        "context": retriver | format_docs, "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [141]:
rag_chain.invoke("whatv is gen ai?")

'<think>\nOkay, the user is asking "what is gen ai?" Looking at the provided context, I need to extract the relevant information. \n\nThe context mentions generative AI as a subset of AI that uses large-language models (LLMs). Examples given include OpenAI’s GPT-4, Google’s Gemini, and Meta’s LLaMA. These models are trained on massive internet-scale databases with billions of parameters to predict the next word in a response.\n\nI should explain that generative AI is part of AI built on LLMs, which are trained on large datasets to generate text. The key points are the components (LLMs), training data, and the purpose of generating human-like text. Also, note the time frame mentioned (June 2022 to June 2023) indicates when these models became prominent.\n\nMake sure to mention the examples provided and the economic impact mentioned in the context, like productivity growth in knowledge-intensive industries. Avoid going into too much detail about the economic studies unless necessary. Kee